# Защита персональных данных клиентов

Требуется защитить данные клиентов страховой компании, разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
display(data)
data.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

В данных отсутствуют пропуски, следовательно можно переходить к этапу машинного обучения.

Страховые выплаты - целевой признак

Пол, Возраст, Зарплата, Члены семьи - признаки

## Алгоритм преобразования

**Алгоритм**

1) Обучение линейной регрессии

2) Создание обратимой матрицы

3) Умножение признаков на эту матрицу

4) Вычисление r2 для новой модели

**Обоснование**

При умножении признаков на обратимую матрицу, сами признаки не меняются; мы переходим в новую систему. Поэтому качество модели не изменится.

## Проверка алгоритма

Проверка модели без умножения на обратимую матрицу:

In [4]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


Проверка модели с умножением на обратимую матрицу:

In [5]:
model = LinearRegression()
matrix = np.random.normal(loc = 0.0, scale = 2.0, size = (4, 4))
np.linalg.inv(matrix)
features = features @ matrix
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249455028665523


Качество модели до и после обучения не изменилось (r2 = 0.425)